In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import time

# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Canvas
canvas = np.zeros((480, 640, 3), np.uint8)

# Drawing variables
px, py = 0, 0
draw_color = (255, 0, 255)
brush_thickness = 7
eraser_thickness = 50

# Button areas
colors = {
    "purple": (255, 0, 255),
    "blue": (255, 0, 0),
    "green": (0, 255, 0),
    "red": (0, 0, 255)
}

def fingers_up(hand):
    tips = [4, 8, 12, 16, 20]
    fingers = []
    fingers.append(hand.landmark[tips[0]].x < hand.landmark[tips[0]-1].x)
    for i in range(1, 5):
        fingers.append(hand.landmark[tips[i]].y < hand.landmark[tips[i]-2].y)
    return fingers

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    # Draw top bar
    cv2.rectangle(frame, (0, 0), (640, 60), (50, 50, 50), -1)
    cv2.putText(frame, "PURPLE", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,255), 2)
    cv2.putText(frame, "BLUE", (120, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)
    cv2.putText(frame, "GREEN", (220, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    cv2.putText(frame, "RED", (350, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    cv2.putText(frame, "CLEAR", (470, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

    if result.multi_hand_landmarks:
        for hand in result.multi_hand_landmarks:
            lm = hand.landmark
            h, w, _ = frame.shape
            x1, y1 = int(lm[8].x * w), int(lm[8].y * h)
            x2, y2 = int(lm[12].x * w), int(lm[12].y * h)

            finger_status = fingers_up(hand)

            # Selection mode (two fingers)
            if finger_status[1] and finger_status[2]:
                px, py = 0, 0
                if y1 < 60:
                    if 0 < x1 < 100:
                        draw_color = colors["purple"]
                    elif 100 < x1 < 200:
                        draw_color = colors["blue"]
                    elif 200 < x1 < 330:
                        draw_color = colors["green"]
                    elif 330 < x1 < 460:
                        draw_color = colors["red"]
                    elif 460 < x1 < 640:
                        canvas = np.zeros((480, 640, 3), np.uint8)

                cv2.rectangle(frame, (x1-15, y1-15), (x2+15, y2+15), draw_color, 2)

            # Drawing mode (index finger)
            elif finger_status[1] and not finger_status[2]:
                if px == 0 and py == 0:
                    px, py = x1, y1
                cv2.line(canvas, (px, py), (x1, y1), draw_color, brush_thickness)
                px, py = x1, y1

            # Eraser mode (fist)
            elif not any(finger_status):
                cv2.line(canvas, (px, py), (x1, y1), (0, 0, 0), eraser_thickness)
                px, py = x1, y1

            mp_draw.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)
    else:
        px, py = 0, 0

    # Merge canvas and frame
    gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
    _, inv = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY_INV)
    inv = cv2.cvtColor(inv, cv2.COLOR_GRAY2BGR)
    frame = cv2.bitwise_and(frame, inv)
    frame = cv2.bitwise_or(frame, canvas)

    cv2.imshow("Air Canvas", frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    if key == ord('s'):
        cv2.imwrite(f"air_canvas_{int(time.time())}.png", canvas)

cap.release()
cv2.destroyAllWindows()